## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145655 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [2]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving resnet_builder.py to resnet_builder.py
User uploaded file "resnet_builder.py" with length 5409 bytes


In [0]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras import models, layers
from keras.optimizers import Adadelta, Adam, RMSprop
from keras.utils import to_categorical
from keras.layers import Flatten, Dense, GlobalAveragePooling2D, Dropout
from keras.applications.resnet50 import ResNet50

BATCH_SIZE = 32 # batch 的大小，如果出現 OOM error，請降低這個值
NUM_CLASSES = 10 # 類別的數量，Cifar 10 共有 10 個類別
EPOCHS = 30 # 訓練整個資料集共 30個循環
IMAGE_SIZE = (32, 32) # 影像大小
FREEZE_LAYERS = 1 # 凍結網路層數

In [26]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
#train_data_generator = ImageDataGenerator(
#    rotation_range=40,
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    shear_range=0.2,
#    zoom_range=0.2,
#    channel_shift_range=10,
#    horizontal_flip=True,
#    fill_mode='nearest')

#test_data_generator = ImageDataGenerator()

#train_data_imgs = train_data_generator.flow(x_train, y_train, batch_size=BATCH_SIZE, shuffle=False)
#test_data_imgs = test_data_generator.flow(x_test, y_test, batch_size=BATCH_SIZE, shuffle=False)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [19]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3), depth=56)
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_442 (Activation)     (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [20]:
#batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
#num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
#epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 153s 3ms/step - loss: 2.2367 - acc: 0.5196 - val_loss: 2.4361 - val_acc: 0.4582
Epoch 2/30
50000/50000 [==============================] - 126s 3ms/step - loss: 1.5670 - acc: 0.6748 - val_loss: 1.6518 - val_acc: 0.6201
Epoch 3/30
50000/50000 [==============================] - 125s 3ms/step - loss: 1.2576 - acc: 0.7319 - val_loss: 1.5900 - val_acc: 0.6180
Epoch 4/30
50000/50000 [==============================] - 127s 3ms/step - loss: 1.0733 - acc: 0.7719 - val_loss: 1.2034 - val_acc: 0.7274
Epoch 5/30
50000/50000 [==============================] - 125s 2ms/step - loss: 0.9548 - acc: 0.7986 - val_loss: 1.3070 - val_acc: 0.6833
Epoch 6/30
50000/50000 [==============================] - 125s 2ms/step - loss: 0.8727 - acc: 0.8174 - val_loss: 1.7894 - val_acc: 0.5955
Epoch 7/30
50000/50000 [==============================] - 126s 3ms/step - loss: 0.8146 - acc: 0.8351 - val_loss: 1.1949 - 

In [28]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
conv_base = ResNet50(include_top=False, weights='imagenet', 
               input_shape=(200, 200, 3))

model = models.Sequential()
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(layers.UpSampling2D((2,2)))
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

# 增加 DropOut layer
#x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
#output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

#net_final = Model(inputs=model.input, outputs=output_layer)

# 設定凍結與要進行訓練的網路層
#for layer in net_final.layers[:FREEZE_LAYERS]:
#    layer.trainable = False
#for layer in net_final.layers[FREEZE_LAYERS:]:
#    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
#net_final.compile(optimizer=Adam(lr=1e-5),
#                  loss='categorical_crossentropy', metrics=['accuracy'])

#net_final.summary()# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
#model = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
#               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

#x = model.output
#x = Flatten()(x)

# 增加 DropOut layer
#x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
#output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

#net_final = Model(inputs=model.input, outputs=output_layer)

# 設定凍結與要進行訓練的網路層
#for layer in net_final.layers[:FREEZE_LAYERS]:
#    layer.trainable = False
#for layer in net_final.layers[FREEZE_LAYERS:]:
#    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
#net_final.compile(optimizer=Adam(lr=1e-5),
#                  loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [30]:
#model.compile(loss='categorical_crossentropy',
#              optimizer=Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
#              metrics=['accuracy'])

model.compile(optimizer=RMSprop(lr=2e-5), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 480s 10ms/step - loss: 0.2300 - acc: 0.9179 - val_loss: 0.1121 - val_acc: 0.9598
Epoch 2/30
50000/50000 [==============================] - 453s 9ms/step - loss: 0.1380 - acc: 0.9506 - val_loss: 0.0700 - val_acc: 0.9779
Epoch 3/30
50000/50000 [==============================] - 453s 9ms/step - loss: 0.0990 - acc: 0.9671 - val_loss: 0.0495 - val_acc: 0.9852
Epoch 4/30
50000/50000 [==============================] - 453s 9ms/step - loss: 0.0757 - acc: 0.9766 - val_loss: 0.0429 - val_acc: 0.9867
Epoch 5/30
50000/50000 [==============================] - 452s 9ms/step - loss: 0.0585 - acc: 0.9836 - val_loss: 0.0373 - val_acc: 0.9882
Epoch 6/30
50000/50000 [==============================] - 452s 9ms/step - loss: 0.0455 - acc: 0.9881 - val_loss: 0.0365 - val_acc: 0.9883
Epoch 7/30
50000/50000 [==============================] - 452s 9ms/step - loss: 0.0366 - acc: 0.9911 - val_loss: 0.0336 

In [0]:
model.summary()